In [43]:
import foolbox as fb
import foolbox.attacks as fa
import numpy as np
import torch
from torch import nn

weight = torch.load('./weight/ae_for_number_{}.pth'.format(1),map_location=torch.device('cpu'))

In [12]:
weight.keys()

odict_keys(['encoder.0.weight', 'encoder.0.bias', 'encoder.2.weight', 'encoder.2.bias', 'encoder.4.weight', 'encoder.4.bias', 'encoder.6.weight', 'encoder.6.bias', 'decoder.0.weight', 'decoder.0.bias', 'decoder.2.weight', 'decoder.2.bias', 'decoder.4.weight', 'decoder.4.bias', 'decoder.6.weight', 'decoder.6.bias'])

In [140]:
classNum = 10
from copy import deepcopy
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class integratedAutoencoder(nn.Module):
    def __init__(self):
        super(integratedAutoencoder, self).__init__()
        for net_index in range(10):
            exec("self.network{}=autoencoder()".format(net_index))
            exec("self.network{0}.load_state_dict(torch.load('./weight/ae_for_number_{1}.pth',map_location=torch.device('cpu')))".format(net_index,net_index+1))
    def forward(self,x):
        output = []
        for net_index in range(10):
            temp=deepcopy(x)
            exec("output.append(self.network{}(temp))".format(net_index))
        #sum each piexl in each pic in output to get the overall brightness.
        output=[torch.sum(pic/2+0.5) for pic in output]
        return torch.tensor(output.index(max(output)))
intAE = integratedAutoencoder()
intAE.eval()
fmodel = fb.PyTorchModel(intAE,bounds=(0,1))

In [143]:
images, labels = fb.utils.samples(fmodel, dataset='MNIST',batchsize=128) 
# change image in images to 784 rows vector
imagesVectors =torch.zeros(images.shape[0],images.shape[2]*images.shape[3])
for k in range(images.shape[0]):
    n=0
    for i in range(28):
        for j in range(28):
            imagesVectors[k,n]=images[k,0,i,j]
            n+=1
images = imagesVectors

In [145]:
# design an input that looks like a 1
x = np.zeros((28, 28), dtype=np.float32)
x[5:-5, 12:-12] = 1
x=torch.from_numpy(x)
x = x.view(784)#change shape to vector
output = intAE(x)

In [148]:
fb.utils.accuracy(fmodel, images, labels)

0.046875

In [149]:
metric = fb.distances.LpDistance(0)
attack = fa.SaltAndPepperNoiseAttack()
A = attack(fmodel, images, labels, epsilons=0.03)
#raw, clipped, is_adv 

AssertionError: 

In [78]:
labels

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 9, 2, 1, 1,
        6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 9, 2, 1, 1, 6, 1, 4, 6,
        5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5,
        7, 3, 4, 1, 2, 4, 8, 0, 9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1,
        2, 4, 8, 0, 9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0,
        9, 2, 1, 1, 6, 1, 4, 6])

In [76]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

testset = MNIST('./data', download=True, train=False)
testloader = DataLoader(testset, batch_size=128, shuffle=True)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>